In [3]:
import json
import numpy as np
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
from gensim.models import Word2Vec
from collections import Counter
from keras.preprocessing.sequence import pad_sequences

from src.service._training_data import strip_accents
from src.utils import ColourConverter

tf.compat.v1.disable_eager_execution()

In [22]:
import numpy as np


def vectorize_words():
    with open(
        '/Users/shreyanshrijan/Documents/colour-text-to-colour/src/train_test_data/universal_data.json'
    ) as f:
        data = f.read()
    data = json.loads(data)

    stop_words = ['a', 'of', 'the', 'an', 'by', 'in', 'at', 'to', 'n', 'no']

    color_data = {strip_accents(
        data[i]['name']
    ).lower(): data[i]['hex'] for i in range(len(data))}

    for k, v in color_data.items():
        color_data[k] = ColourConverter.hex_to_lab(color_data[k])

    col_words = Counter()
    for color_i in color_data.keys():
        col_words.update(color_i.split())

    color_score = lambda color: (
        sum(col_words[w] for w in color.split() if w not in stop_words)
        / sum(1.0 for w in color.split() if w not in stop_words)
    )
    color_data = {k: v for k, v in color_data.items() if color_score(k) > 2}

    col_words = Counter()
    for color_i in color_data.keys():
        col_words.update(w for w in color_i.split() if w not in stop_words)

    colour_list = []
    for k, v in color_data.items():
        colour_list.append(k.split())

    w2v = Word2Vec(colour_list, min_count=1, vector_size=5)

    sequence_length = 4
    vector_dict = {}
    for k, v in color_data.items():
        word_vector = [np.array(w2v.wv[m]) for m in k.split()]
        word_vector = []
        for m in k.split():
            word_vector.append(np.array(w2v.wv[m]))
        
        if len(word_vector) < sequence_length:
            padding = [np.zeros(5)] * (sequence_length - len(word_vector))
            word_vector.extend(padding)
        vector_dict[tf.convert_to_tensor(np.array(word_vector), dtype=tf.float32)] = v

    return vector_dict, w2v

if __name__ == '__main__':
    word_vectors, w2v = vectorize_words()


In [24]:
len(word_vectors)

25921

In [27]:
list(w2v.wv['green'])

[0.19955392, 1.148622, 1.852713, -0.89102215, -0.4622761]